# Visualize simulated dynamics of 3v1 ball possession games
Generate animation videos.

In [1]:
include("../src/BallPossessionModel.jl")
import .BallPossessionModel as BPM

In [2]:
using Random
using Dates
using TimeZones
import Statistics: mean, std
using CairoMakie
using Makie.GeometryBasics
using Printf
using LaTeXStrings

## Utilities

In [3]:
""" Fill unspecified parameter values and return a Parameter object. """
function init_params(; kwargs...)
    d_params = Dict(
        :Δt => 1e-2,
        :maxpass => 20,
        :L => 6,
        :L_out => 0.5,
        :L_buff => 0.5,
        :m => 65,
        :T => 0.71,
        :σ => 6.0,
        :β => 8.0,
        :kr => 200,
        :γ => 400,
        :kf => 100,
        :Lf => 5.7,
        :ke => 100,
        :Le => 3.2,
        :q => 0.45,
        :τ => 1.25,
        :L_ict => 0.65,
    )
    for key in keys(kwargs)
        d_params[key] = kwargs[key]
    end
    return BPM.Parameters(; d_params...)
end

init_params

In [4]:
""" Run a simulation, print relevant infos, and return the Result object. """
function run_sim(seed, p::BPM.Parameters)
    rng = Xoshiro(seed)
    res = BPM.simulate(rng, p)
    println(res.message, " at t = ", round(res.endtime, digits=3))
    println("num. of passes: ", res.passcount)
    area_ts = BPM.get_area_timeseries(res)[1:res.endid]
    println("average OF area: ", round(mean(area_ts), digits=3))
    println("std. of OF area: ", round(std(area_ts), digits=3))
    return res
end

run_sim

In [5]:
function save_animation(
    seed::Integer, p::BPM.Parameters, res::BPM.Result, stepsize::Int;
    showguide=true, saveanim=true,
)
    # get current time andgenerate a file name
    fname = (
        Dates.format(now(tz"Europe/Berlin"), "yymmdd_HHMMSS") *
        "-bp-anim"
    )
    return save_animation(seed, p, res, stepsize; fname=fname, showguide=showguide, saveanim=saveanim)
end

function save_animation(
    seed::Integer, p::BPM.Parameters, res::BPM.Result, stepsize::Int;
    fname::String, showguide=true, saveanim=true,
)
    if showguide
        fname *= "-w_guide"
    else
        fname *= "-wo_guide"
    end
    fname *= ".mp4"  # extension
    # get list of times to update plot
    timestamps = 1:stepsize:length(res.t)
    framerate::Int = round(1 / (stepsize * p.Δt))
    # set up observables
    n = Observable{Int}(1)
    timestr = @lift(latexstring(@sprintf "t = %05.2f" res.t[$n]))
    un = @lift(res.u[:, $n])
    pP = @lift(Point2f($un[5], $un[6]))
    pM = @lift(Point2f($un[7], $un[8]))
    pR = @lift(Point2f($un[9], $un[10]))
    pD = @lift(Point2f($un[11], $un[12]))
    pB = @lift(Point2f($un[13], $un[14]))
    circ_ict = @lift(Circle($pD, p.L_ict))  # intercept area
    ballclr = @lift(res.t[$n] < res.endtime ? :black : :red)  # color of ball
    if showguide
        OFtri = @lift([$pP, $pM, $pR])
        circ_ff = @lift(Circle($pR, p.Lf))
        circ_ef = @lift(Circle($pD, p.Le))
    end
    # prepare helper variables
    halfL = p.L / 2
    outline = p.L / 2 + p.L_out
    xylim = halfL + 2 * p.L_out
    # prepare figure
    f = Figure(size=(600, 450), figure_padding=5)
    # prepare main axis
    ax_main = Axis(f[2, 1], 
        aspect=DataAspect(), alignmode=Outside(), halign=:right,
        xlabel=L"x / \mathrm{m}", ylabel=L"y / \mathrm{m}", xgridvisible=false, ygridvisible=false,
        limits=((-xylim, xylim), (-xylim, xylim)),
    )
    # draw play field
    unfilledpoly = Dict(:color => :transparent, :strokewidth => 2)
    poly!(
        Point2f[(-halfL, -halfL), (halfL, -halfL), (halfL, halfL), (-halfL, halfL)];
        strokecolor=:black, linestyle=:solid, unfilledpoly...,
    )
    poly!(Point2f[
        (-outline, -outline), (outline, -outline), (outline, outline), (-outline, outline)
    ]; strokecolor=:red, linestyle=:dash, unfilledpoly...,
    )
    if showguide
        (inner, outer) = @. (p.L + [-p.L_buff, p.L_buff]) / 2
        pol_buffer = Polygon(
            Point2f[(-outer, -outer), (outer, -outer), (outer, outer), (-outer, outer)],
            [Point2f[(-inner, -inner), (inner, -inner), (inner, inner), (-inner, inner)]],
        )
        poly!(pol_buffer; color=(:black, 0.2))
        # OF triangle
        poly!(OFtri; color=(:cyan, 0.1))
        # natural length for following force
        poly!(circ_ff; 
            strokecolor=:dodgerblue, linestyle=:dot, 
            unfilledpoly...
        )
        # natural length for evading force
        poly!(circ_ef; 
            strokecolor=:tomato, linestyle=:dashdot, 
            unfilledpoly...
        )
    end
    # intercept region around DF
    poly!(circ_ict, color=(:orangered, 0.3))
    # players and ball
    ms = Dict(:markersize => 20)
    scatter!(ax_main, pP; label="Passer", marker='P', color=:deeppink3, ms...)
    scatter!(ax_main, pM; label="Mover", marker='M', color=:darkgreen, ms...)
    scatter!(ax_main, pR; label="Receiver", marker='R', color=:dodgerblue3, ms...)
    scatter!(ax_main, pD; label="Defender", marker='D', color=:maroon, ms...)
    scatter!(ax_main, pB; label="Ball", color=ballclr, markersize=10)
    # put legend
    Legend(f[2, 2], ax_main, 
        labelsize=18, rowgap=10, tellwidth=false, halign=:left
    )
    # annotate time and parameter values
    Label(f[1, 1:2], timestr, fontsize=18)
    Label(
        f[3, 1:2], latexstring(join(BPM.param2strlist(p, seed), ", ")), 
        lineheight=1.2, fontsize=12, word_wrap=true,
    )
    # FOR DEBUG: show panel regions
    @debug Box(f[1, 1:2], color=(:cyan, 0.2), strokewidth=0)
    @debug Box(f[2, 1], color=(:red, 0.2), strokewidth=0)
    @debug Box(f[3, 1:2], color=(:green, 0.2), strokewidth=0)
    # resize axes and figure
    colsize!(f.layout, 1, Relative(0.75))
    # save animation
    video_io = Record(f, timestamps; framerate=framerate) do timestep
        n[] = timestep
    end
    if saveanim
        save(fname, video_io)
    end
    return(video_io)
end

save_animation (generic function with 1 method)

## Generate animations

### Supplementary Video S1: intercept w/ guide

In [6]:
s1 = 2
# inspiration: lower-level team
p1 = init_params(; 
    σ=8.0, β=4, kf=50, Lf=5.3, ke=150, Le=2.9, q=0.3, τ=1.1, L_ict=0.6
)
res1 = run_sim(s1, p1);

intercept at t = 9.62
num. of passes: 13
average OF area: 10.685
std. of OF area: 0.583


In [7]:
# with guide shapes
ani = save_animation(s1, p1, res1, 5, fname="S1_intercept", showguide=true, saveanim=true)

VideoStream(Base.PipeEndpoint(RawFD(-1) closed, 0 bytes waiting), Process(setenv(`/opt/julia/artifacts/fdb14f7b1bf6766687af4e3951c4d982cee66086/bin/ffmpeg -y -loglevel quiet -threads 20 -framerate 20 -pixel_format rgb24 -f rawvideo -s:v 1200x900 -r 20 -i pipe:0 -vf vflip -profile:v high422 -crf 20 -preset slow -c:v libx264 -pix_fmt yuv420p -an '/tmp/jl_8VK8AU/##video#277.mp4'`,["_CE_M=", "PATH=/opt/julia/artifacts/b8a470e4553346122607f94ec60d83c77f061cd6/bin:/opt/julia/artifacts/0ccc8805e5047cf9fbd848c4f7d4c15715aad7c9/bin:/opt/julia/artifacts/96bc7b9ddb319d1a051535c85184aa2bf42dbd8b/bin:/opt/julia/artifacts/bd752d7807f5f10e2dce3d2b2b683589fa03d31a/bin:/opt/julia/artifacts/c8a20a2030f10b70947d8d2a6bff7f8b5f343fe9/bin:/opt/julia/artifacts/0b5269206a6849e48487ed04136c30b1f12443e7/bin:/opt/julia/artifacts/bdb4015047b09c6ac5002f73dfd5be8ddf59189b/bin:/opt/julia/artifacts/00023f2f62a04af7531278ce1a689ff93800ffa1/bin:/opt/julia/artifacts/c951fb23b5652def1dea483af7095a38f3b3cefd/bin:/opt/julia/artifacts/fdb14f7b1bf6766687af4e3951c4d982cee66086/bin:/vscode/vscode-server/bin/linux-x64/1a5daa3a0231a0fbba4f14db7ec463cf99d7768e/bin/remote-cli:/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin", "REMOTE_CONTAINERS_DISPLAY_SOCK=/tmp/.X11-unix/X3", "VSCODE_HANDLES_SIGPIPE=true", "ELECTRON_RUN_AS_NODE=1", "WAYLAND_DISPLAY=vscode-wayland-1dd6f3b4-b200-41a9-8238-d0e021c5be31.sock", "VSCODE_NLS_CONFIG={\"locale\":\"en\",\"osLocale\":\"en\",\"availableLanguages\":{}}", "CONDA_PYTHON_EXE=/opt/conda/bin/python", "RSTUDIO_WHICH_R=/opt/conda/bin/R", "LC_ALL=C.UTF-8"  …  "NB_GID=100", "REMOTE_CONTAINERS_SOCKETS=[\"/tmp/vscode-ssh-auth-b2f7ea7f-9298-48f3-acfd-ab22d34eadd2.sock\",\"/tmp/.X11-unix/X3\"]", "_CE_CONDA=", "CONDA_SHLVL=1", "REMOTE_CONTAINERS_IPC=/tmp/vscode-remote-containers-ipc-b2f7ea7f-9298-48f3-acfd-ab22d34eadd2.sock", "CONDA_EXE=/opt/conda/bin/conda", "HOME=/home/jovyan", "LS_COLORS=", "HOSTNAME=29c8278517d0", "OPENBLAS_MAIN_FREE=1"]), ProcessExited(0)), CairoMakie.Screen{CairoMakie.IMAGE}(Scene (600px, 450px):
  0 Plots
  4 Child Scenes:
    ├ Scene (600px, 450px)
    ├ Scene (600px, 450px)
    ├ Scene (600px, 450px)
    └ Scene (600px, 450px), Cairo.CairoSurfaceImage{ColorTypes.ARGB32}(Ptr{Nothing} @0x000000002d8766b0, 1200.0, 900.0, ColorTypes.ARGB32[ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); … ; ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8)]), Cairo.CairoContext(Ptr{Nothing} @0x000000002a8a3d50, Cairo.CairoSurfaceImage{ColorTypes.ARGB32}(Ptr{Nothing} @0x000000002d8766b0, 1200.0, 900.0, ColorTypes.ARGB32[ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); … ; ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8)]), Ptr{Nothing} @0x000000002d1166b0), 2.0, 6, false, CairoMakie.ScreenConfig(2.0, 0.75, :best, false, false, nothing)), Makie.TickController(Observable(Makie.Tick(Makie.OneTimeRende

### Supplementary Video S2: maximum pass w/o guide

In [8]:
s2 = 7
# inspiration: high-level team
p2 = init_params(; 
    σ=6.0, β=8, kf=150, Lf=5.7, ke=50, Le=3.2, q=0.45, τ=0.9, L_ict=0.65
)
res2 = run_sim(s2, p2);

max_pass at t = 14.2
num. of passes: 20
average OF area: 12.906
std. of OF area: 0.632


In [9]:
# with guide shapes
ani = save_animation(s2, p2, res2, 5, fname="S2_maxpass", showguide=false, saveanim=true)

VideoStream(Base.PipeEndpoint(RawFD(-1) closed, 0 bytes waiting), Process(setenv(`/opt/julia/artifacts/fdb14f7b1bf6766687af4e3951c4d982cee66086/bin/ffmpeg -y -loglevel quiet -threads 20 -framerate 20 -pixel_format rgb24 -f rawvideo -s:v 1200x900 -r 20 -i pipe:0 -vf vflip -profile:v high422 -crf 20 -preset slow -c:v libx264 -pix_fmt yuv420p -an '/tmp/jl_iDiUTA/##video#278.mp4'`,["_CE_M=", "PATH=/opt/julia/artifacts/b8a470e4553346122607f94ec60d83c77f061cd6/bin:/opt/julia/artifacts/0ccc8805e5047cf9fbd848c4f7d4c15715aad7c9/bin:/opt/julia/artifacts/96bc7b9ddb319d1a051535c85184aa2bf42dbd8b/bin:/opt/julia/artifacts/bd752d7807f5f10e2dce3d2b2b683589fa03d31a/bin:/opt/julia/artifacts/c8a20a2030f10b70947d8d2a6bff7f8b5f343fe9/bin:/opt/julia/artifacts/0b5269206a6849e48487ed04136c30b1f12443e7/bin:/opt/julia/artifacts/bdb4015047b09c6ac5002f73dfd5be8ddf59189b/bin:/opt/julia/artifacts/00023f2f62a04af7531278ce1a689ff93800ffa1/bin:/opt/julia/artifacts/c951fb23b5652def1dea483af7095a38f3b3cefd/bin:/opt/julia/artifacts/fdb14f7b1bf6766687af4e3951c4d982cee66086/bin:/vscode/vscode-server/bin/linux-x64/1a5daa3a0231a0fbba4f14db7ec463cf99d7768e/bin/remote-cli:/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin", "REMOTE_CONTAINERS_DISPLAY_SOCK=/tmp/.X11-unix/X3", "VSCODE_HANDLES_SIGPIPE=true", "ELECTRON_RUN_AS_NODE=1", "WAYLAND_DISPLAY=vscode-wayland-1dd6f3b4-b200-41a9-8238-d0e021c5be31.sock", "VSCODE_NLS_CONFIG={\"locale\":\"en\",\"osLocale\":\"en\",\"availableLanguages\":{}}", "CONDA_PYTHON_EXE=/opt/conda/bin/python", "RSTUDIO_WHICH_R=/opt/conda/bin/R", "LC_ALL=C.UTF-8"  …  "NB_GID=100", "REMOTE_CONTAINERS_SOCKETS=[\"/tmp/vscode-ssh-auth-b2f7ea7f-9298-48f3-acfd-ab22d34eadd2.sock\",\"/tmp/.X11-unix/X3\"]", "_CE_CONDA=", "CONDA_SHLVL=1", "REMOTE_CONTAINERS_IPC=/tmp/vscode-remote-containers-ipc-b2f7ea7f-9298-48f3-acfd-ab22d34eadd2.sock", "CONDA_EXE=/opt/conda/bin/conda", "HOME=/home/jovyan", "LS_COLORS=", "HOSTNAME=29c8278517d0", "OPENBLAS_MAIN_FREE=1"]), ProcessExited(0)), CairoMakie.Screen{CairoMakie.IMAGE}(Scene (600px, 450px):
  0 Plots
  4 Child Scenes:
    ├ Scene (600px, 450px)
    ├ Scene (600px, 450px)
    ├ Scene (600px, 450px)
    └ Scene (600px, 450px), Cairo.CairoSurfaceImage{ColorTypes.ARGB32}(Ptr{Nothing} @0x000000002cffd870, 1200.0, 900.0, ColorTypes.ARGB32[ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); … ; ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8)]), Cairo.CairoContext(Ptr{Nothing} @0x00000000208de250, Cairo.CairoSurfaceImage{ColorTypes.ARGB32}(Ptr{Nothing} @0x000000002cffd870, 1200.0, 900.0, ColorTypes.ARGB32[ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); … ; ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8)]), Ptr{Nothing} @0x000000001cf18410), 2.0, 6, false, CairoMakie.ScreenConfig(2.0, 0.75, :best, false, false, nothing)), Makie.TickController(Observable(Makie.Tick(Makie.OneTimeRende

### Supplementary Video S3: ball-out w/o guide

In [10]:
s3 = 4
# inspiration: high-level team
p3 = init_params(; 
    σ=6.0, β=8, kf=150, Lf=5.7, ke=50, Le=3.2, q=0.45, τ=0.9, L_ict=0.65
)
res3 = run_sim(s3, p3);

ball_out at t = 12.06
num. of passes: 16
average OF area: 12.69
std. of OF area: 0.525


In [11]:
# without guide shapes
ani = save_animation(s3, p3, res3, 5, fname="S3_ballout", showguide=false, saveanim=true)

VideoStream(Base.PipeEndpoint(RawFD(-1) closed, 0 bytes waiting), Process(setenv(`/opt/julia/artifacts/fdb14f7b1bf6766687af4e3951c4d982cee66086/bin/ffmpeg -y -loglevel quiet -threads 20 -framerate 20 -pixel_format rgb24 -f rawvideo -s:v 1200x900 -r 20 -i pipe:0 -vf vflip -profile:v high422 -crf 20 -preset slow -c:v libx264 -pix_fmt yuv420p -an '/tmp/jl_iC3GnR/##video#279.mp4'`,["_CE_M=", "PATH=/opt/julia/artifacts/b8a470e4553346122607f94ec60d83c77f061cd6/bin:/opt/julia/artifacts/0ccc8805e5047cf9fbd848c4f7d4c15715aad7c9/bin:/opt/julia/artifacts/96bc7b9ddb319d1a051535c85184aa2bf42dbd8b/bin:/opt/julia/artifacts/bd752d7807f5f10e2dce3d2b2b683589fa03d31a/bin:/opt/julia/artifacts/c8a20a2030f10b70947d8d2a6bff7f8b5f343fe9/bin:/opt/julia/artifacts/0b5269206a6849e48487ed04136c30b1f12443e7/bin:/opt/julia/artifacts/bdb4015047b09c6ac5002f73dfd5be8ddf59189b/bin:/opt/julia/artifacts/00023f2f62a04af7531278ce1a689ff93800ffa1/bin:/opt/julia/artifacts/c951fb23b5652def1dea483af7095a38f3b3cefd/bin:/opt/julia/artifacts/fdb14f7b1bf6766687af4e3951c4d982cee66086/bin:/vscode/vscode-server/bin/linux-x64/1a5daa3a0231a0fbba4f14db7ec463cf99d7768e/bin/remote-cli:/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin", "REMOTE_CONTAINERS_DISPLAY_SOCK=/tmp/.X11-unix/X3", "VSCODE_HANDLES_SIGPIPE=true", "ELECTRON_RUN_AS_NODE=1", "WAYLAND_DISPLAY=vscode-wayland-1dd6f3b4-b200-41a9-8238-d0e021c5be31.sock", "VSCODE_NLS_CONFIG={\"locale\":\"en\",\"osLocale\":\"en\",\"availableLanguages\":{}}", "CONDA_PYTHON_EXE=/opt/conda/bin/python", "RSTUDIO_WHICH_R=/opt/conda/bin/R", "LC_ALL=C.UTF-8"  …  "NB_GID=100", "REMOTE_CONTAINERS_SOCKETS=[\"/tmp/vscode-ssh-auth-b2f7ea7f-9298-48f3-acfd-ab22d34eadd2.sock\",\"/tmp/.X11-unix/X3\"]", "_CE_CONDA=", "CONDA_SHLVL=1", "REMOTE_CONTAINERS_IPC=/tmp/vscode-remote-containers-ipc-b2f7ea7f-9298-48f3-acfd-ab22d34eadd2.sock", "CONDA_EXE=/opt/conda/bin/conda", "HOME=/home/jovyan", "LS_COLORS=", "HOSTNAME=29c8278517d0", "OPENBLAS_MAIN_FREE=1"]), ProcessExited(0)), CairoMakie.Screen{CairoMakie.IMAGE}(Scene (600px, 450px):
  0 Plots
  4 Child Scenes:
    ├ Scene (600px, 450px)
    ├ Scene (600px, 450px)
    ├ Scene (600px, 450px)
    └ Scene (600px, 450px), Cairo.CairoSurfaceImage{ColorTypes.ARGB32}(Ptr{Nothing} @0x0000000027da4510, 1200.0, 900.0, ColorTypes.ARGB32[ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); … ; ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8)]), Cairo.CairoContext(Ptr{Nothing} @0x0000000024625e50, Cairo.CairoSurfaceImage{ColorTypes.ARGB32}(Ptr{Nothing} @0x0000000027da4510, 1200.0, 900.0, ColorTypes.ARGB32[ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); … ; ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8)]), Ptr{Nothing} @0x000000001d147010), 2.0, 6, false, CairoMakie.ScreenConfig(2.0, 0.75, :best, false, false, nothing)), Makie.TickController(Observable(Makie.Tick(Makie.OneTimeRende

### Supplementary Video S4: intercept w/o guide

In [12]:
s4 = 3
# inspiration: lower-level team
p4 = init_params(; 
    σ=8.0, β=4, kf=50, Lf=5.3, ke=150, Le=2.9, q=0.3, τ=1.1, L_ict=0.6
)
res4 = run_sim(s4, p4);

intercept at t = 8.88
num. of passes: 12
average OF area: 10.343
std. of OF area: 0.759


In [13]:
# with guide shapes
ani = save_animation(s4, p4, res4, 5, fname="S4_intercept", showguide=false, saveanim=true)

VideoStream(Base.PipeEndpoint(RawFD(-1) closed, 0 bytes waiting), Process(setenv(`/opt/julia/artifacts/fdb14f7b1bf6766687af4e3951c4d982cee66086/bin/ffmpeg -y -loglevel quiet -threads 20 -framerate 20 -pixel_format rgb24 -f rawvideo -s:v 1200x900 -r 20 -i pipe:0 -vf vflip -profile:v high422 -crf 20 -preset slow -c:v libx264 -pix_fmt yuv420p -an '/tmp/jl_4DWHdl/##video#280.mp4'`,["_CE_M=", "PATH=/opt/julia/artifacts/b8a470e4553346122607f94ec60d83c77f061cd6/bin:/opt/julia/artifacts/0ccc8805e5047cf9fbd848c4f7d4c15715aad7c9/bin:/opt/julia/artifacts/96bc7b9ddb319d1a051535c85184aa2bf42dbd8b/bin:/opt/julia/artifacts/bd752d7807f5f10e2dce3d2b2b683589fa03d31a/bin:/opt/julia/artifacts/c8a20a2030f10b70947d8d2a6bff7f8b5f343fe9/bin:/opt/julia/artifacts/0b5269206a6849e48487ed04136c30b1f12443e7/bin:/opt/julia/artifacts/bdb4015047b09c6ac5002f73dfd5be8ddf59189b/bin:/opt/julia/artifacts/00023f2f62a04af7531278ce1a689ff93800ffa1/bin:/opt/julia/artifacts/c951fb23b5652def1dea483af7095a38f3b3cefd/bin:/opt/julia/artifacts/fdb14f7b1bf6766687af4e3951c4d982cee66086/bin:/vscode/vscode-server/bin/linux-x64/1a5daa3a0231a0fbba4f14db7ec463cf99d7768e/bin/remote-cli:/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin", "REMOTE_CONTAINERS_DISPLAY_SOCK=/tmp/.X11-unix/X3", "VSCODE_HANDLES_SIGPIPE=true", "ELECTRON_RUN_AS_NODE=1", "WAYLAND_DISPLAY=vscode-wayland-1dd6f3b4-b200-41a9-8238-d0e021c5be31.sock", "VSCODE_NLS_CONFIG={\"locale\":\"en\",\"osLocale\":\"en\",\"availableLanguages\":{}}", "CONDA_PYTHON_EXE=/opt/conda/bin/python", "RSTUDIO_WHICH_R=/opt/conda/bin/R", "LC_ALL=C.UTF-8"  …  "NB_GID=100", "REMOTE_CONTAINERS_SOCKETS=[\"/tmp/vscode-ssh-auth-b2f7ea7f-9298-48f3-acfd-ab22d34eadd2.sock\",\"/tmp/.X11-unix/X3\"]", "_CE_CONDA=", "CONDA_SHLVL=1", "REMOTE_CONTAINERS_IPC=/tmp/vscode-remote-containers-ipc-b2f7ea7f-9298-48f3-acfd-ab22d34eadd2.sock", "CONDA_EXE=/opt/conda/bin/conda", "HOME=/home/jovyan", "LS_COLORS=", "HOSTNAME=29c8278517d0", "OPENBLAS_MAIN_FREE=1"]), ProcessExited(0)), CairoMakie.Screen{CairoMakie.IMAGE}(Scene (600px, 450px):
  0 Plots
  4 Child Scenes:
    ├ Scene (600px, 450px)
    ├ Scene (600px, 450px)
    ├ Scene (600px, 450px)
    └ Scene (600px, 450px), Cairo.CairoSurfaceImage{ColorTypes.ARGB32}(Ptr{Nothing} @0x000000002cf71d40, 1200.0, 900.0, ColorTypes.ARGB32[ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); … ; ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8)]), Cairo.CairoContext(Ptr{Nothing} @0x000000002a41f090, Cairo.CairoSurfaceImage{ColorTypes.ARGB32}(Ptr{Nothing} @0x000000002cf71d40, 1200.0, 900.0, ColorTypes.ARGB32[ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); … ; ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8)]), Ptr{Nothing} @0x000000002a75ff70), 2.0, 6, false, CairoMakie.ScreenConfig(2.0, 0.75, :best, false, false, nothing)), Makie.TickController(Observable(Makie.Tick(Makie.OneTimeRende